In [1]:
import pandas as pd
import numpy as np
import pickle
import gc
import os
import time
import copy
import multiprocessing as mp
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import roc_auc_score, log_loss
from tqdm import tqdm
from scipy import sparse, spatial
import warnings
warnings.filterwarnings('ignore')
tqdm.pandas()

/root/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:17: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, defaultdict


In [2]:
import os
import tensorflow as tf

os.environ['CUDA_VISIBLE_DEVICES'] = "0"
# config = tf.compat.v1.ConfigProto()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
# session = tf.compat.v1.Session(config=config)
session = tf.Session(config=config)

from keras.models import Sequential, Model
from keras.layers import Input, Dense, CuDNNLSTM, Bidirectional, Embedding, CuDNNGRU, Conv1D, MaxPooling1D, AveragePooling1D, GlobalMaxPooling1D
from keras.layers import Flatten, PReLU, Dropout, BatchNormalization, SpatialDropout1D, concatenate, Concatenate
from keras.layers import Embedding, Dense, Dropout, Bidirectional, CuDNNLSTM
from keras.layers import Add, Subtract, Multiply, Dot
from attention import Attention
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing.sequence import pad_sequences
from keras.utils import Sequence
from keras.callbacks import Callback
from keras import backend as K
from keras.engine.topology import Layer 
# from keras.utils.training_utils import multi_gpu_model

Using TensorFlow backend.


In [3]:
tic = time.time()
DATA_PATH = '../pkl/'
SAVE_PATH = './feats/'
WEIGHT_PATH = './weights/'
if not os.path.exists(SAVE_PATH):
    print('create dir: %s' % SAVE_PATH)
    os.mkdir(SAVE_PATH)
if not os.path.exists(WEIGHT_PATH):
    print('create dir: %s' % WEIGHT_PATH)
    os.mkdir(WEIGHT_PATH)

print('ques_user_title_desc_textcnn_enc...')

ques_user_title_desc_textcnn_enc...


In [4]:
data = pd.read_pickle(os.path.join(DATA_PATH, 'invite_data.pkl'))
prev_ans_ques_title_desc = pd.read_pickle(os.path.join(DATA_PATH, 'prev_ans_ques_title_desc.pkl'))
word = pd.read_pickle(os.path.join(DATA_PATH, 'word.pkl'))
len_train = (~data['label'].isnull()).sum()
print(len_train)

9489162


In [5]:
data.head()

,iday,ihour,itime,label,qid,uid
0,3865,22,92782,0.0,2166419046,401693808
1,3844,11,92267,0.0,1550017551,3392373099
2,3862,15,92703,0.0,604029601,2317670257
3,3849,11,92387,0.0,2350061229,1618461867
4,3867,4,92812,0.0,2443223942,3544409350


In [6]:
prev_ans_ques_title_desc.head()

,title_q,desc_q,title_u,desc_u
0,"[11058, 272, 2202, 431, 951, 243, 3828, 3037, ...","[3828, 14870, 3037, 2450, 692, 1688, 53547, 24...",[0],[0]
1,"[149, 79, 5210, 22869]","[429, 2709, 1465, 6965]",[0],[0]
2,"[31489, 9218, 3440, 243]","[3046, 221, 3440, 730, 31489, 9218, 1278, 415]","[1116, 6761, 17922, 272, 263, 4534, 7742]","[0, 272, 4534, 7742, 3738, 1277, 111, 2425, 41..."
3,"[973, 64958, 8583, 2269, 628, 565, 2200, 590, ...",[0],[0],[0]
4,"[554, 28208, 396, 51]",[0],[0],[0]


In [7]:
data = pd.concat([data, prev_ans_ques_title_desc], axis=1)
data.head()

,iday,ihour,itime,label,qid,uid,title_q,desc_q,title_u,desc_u
0,3865,22,92782,0.0,2166419046,401693808,"[11058, 272, 2202, 431, 951, 243, 3828, 3037, ...","[3828, 14870, 3037, 2450, 692, 1688, 53547, 24...",[0],[0]
1,3844,11,92267,0.0,1550017551,3392373099,"[149, 79, 5210, 22869]","[429, 2709, 1465, 6965]",[0],[0]
2,3862,15,92703,0.0,604029601,2317670257,"[31489, 9218, 3440, 243]","[3046, 221, 3440, 730, 31489, 9218, 1278, 415]","[1116, 6761, 17922, 272, 263, 4534, 7742]","[0, 272, 4534, 7742, 3738, 1277, 111, 2425, 41..."
3,3849,11,92387,0.0,2350061229,1618461867,"[973, 64958, 8583, 2269, 628, 565, 2200, 590, ...",[0],[0],[0]
4,3867,4,92812,0.0,2443223942,3544409350,"[554, 28208, 396, 51]",[0],[0],[0]


In [8]:
max_len_tq = data['title_q'].apply(len).max()
max_len_dq = data['desc_q'].apply(len).max()
max_len_tu = data['title_u'].apply(len).max()
max_len_du = data['desc_u'].apply(len).max()
print(max_len_tq, max_len_dq) # 110
print(max_len_tu, max_len_du) # 110

38 973
64 256


In [9]:
max_len_tq = 38
max_len_dq = 256
max_len_tu = 64
max_len_du = 256

In [10]:
np.random.seed(42)
def shuffle(l):
    l = l[:256]
    np.random.shuffle(l)
    return l

data['title_q'] = data['title_q'].progress_apply(shuffle)
data['desc_q'] = data['desc_q'].progress_apply(shuffle)
data['title_u'] = data['title_u'].progress_apply(shuffle)
data['desc_u'] = data['desc_u'].progress_apply(shuffle)

100%|██████████| 10630845/10630845 [00:50<00:00, 211358.72it/s]


In [11]:
data.head()

,iday,ihour,itime,label,qid,uid,title_q,desc_q,title_u,desc_u
0,3865,22,92782,0.0,2166419046,401693808,"[3037, 272, 243, 11058, 263, 2202, 951, 431, 3...","[11237, 2202, 263, 33060, 1619, 1688, 272, 245...",[0],[0]
1,3844,11,92267,0.0,1550017551,3392373099,"[79, 149, 22869, 5210]","[2709, 1465, 429, 6965]",[0],[0]
2,3862,15,92703,0.0,604029601,2317670257,"[243, 9218, 31489, 3440]","[221, 3440, 415, 9218, 31489, 3046, 730, 1278]","[7742, 272, 6761, 17922, 4534, 1116, 263]","[4747, 1417, 262, 3093, 3075, 19565, 7742, 152..."
3,3849,11,92387,0.0,2350061229,1618461867,"[973, 2200, 2269, 64958, 628, 565, 8583, 483, ...",[0],[0],[0]
4,3867,4,92812,0.0,2443223942,3544409350,"[51, 396, 554, 28208]",[0],[0],[0]


In [12]:
title_q = pad_sequences(data['title_q'], maxlen=max_len_tq)
desc_q = pad_sequences(data['desc_q'], maxlen=max_len_dq)
title_u = pad_sequences(data['title_u'], maxlen=max_len_tu)
desc_u = pad_sequences(data['desc_u'], maxlen=max_len_du)

In [13]:
title_q.shape

(10630845, 38)

In [14]:
desc_q.shape

(10630845, 256)

In [15]:
# embed_weights = np.array([[0] * 64] + [list(v) for v in word['vector'].values])
embed_weights = np.vstack([np.zeros((1, 64))] + list(word['vector'].values))

In [16]:
embed_weights.shape

(1762830, 64)

In [17]:
del prev_ans_ques_title_desc
gc.collect()

80

In [18]:
class DataSequence(Sequence):
    
    def __init__(self, tq, tu, dq, du, y, batch_size=128):
        self.tq = tq
        self.tu = tu
        self.dq = dq
        self.du = du
        self.y = y
        self.batch_size = batch_size
         
    def __len__(self):
        return int(np.ceil(self.tq.shape[0] / float(self.batch_size)))
    
    def __getitem__(self, idx):
        batch_idx = np.arange(idx * self.batch_size, min((idx + 1) * self.batch_size, self.tq.shape[0]))
        batch_tq = self.tq[batch_idx]
        batch_tu = self.tu[batch_idx]
        batch_dq = self.dq[batch_idx]
        batch_du = self.du[batch_idx]
        batch_y = self.y[batch_idx]
        return [batch_tq, batch_tu, batch_dq, batch_du], batch_y

In [19]:
class MetricsCallback(Callback):
    def __init__(self, trn_tq, trn_tu, trn_dq, trn_du, y_trn,
                       val_tq, val_tu, val_dq, val_du, y_val, 
                       batch_size=128, save_name='weight.h5'):
        self.trn_generator = DataSequence(trn_tq, trn_tu, trn_dq, trn_du, y_trn, BATCH_SIZE)
        self.val_generator = DataSequence(val_tq, val_tu, val_dq, val_du, y_val, BATCH_SIZE)
        self.y_trn = y_trn
        self.y_val = y_val
        self.save_name = save_name
        self.best_score = 0.5

    def on_epoch_end(self, epoch, logs={}):
        # eval train
        y_pred = self.model.predict_generator(self.trn_generator, 
                                              max_queue_size=10, 
                                              workers=1, 
                                              use_multiprocessing=False, 
                                              verbose=0)
        roc = roc_auc_score(self.y_trn, y_pred)
        # eval valid
        y_pred_val = self.model.predict_generator(self.val_generator, 
                                              max_queue_size=10, 
                                              workers=1, 
                                              use_multiprocessing=False, 
                                              verbose=0)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        print('\rroc-auc: %s - roc-auc_val: %s' % (str(round(roc,4)),str(round(roc_val,4))),end=100*' '+'\n')
#         print(roc_val)
#         print(log_loss(self.y_val, y_pred_val)/len(y_pred_val))
        if roc_val > self.best_score:
            self.best_score = roc_val
            self.model.save_weights(os.path.join(WEIGHT_PATH, self.save_name))
        
        return

In [20]:
def Net():
    # title q
    inp_tq = Input(shape=(max_len_tq,))
    x1 = Embedding(embed_weights.shape[0], embed_weights.shape[1], weights=[embed_weights], trainable=False)(inp_tq)
    convs = []
    for kernel_size in [3, 4, 5]:
        c = Conv1D(128, kernel_size, activation='relu')(x1)
        c = GlobalMaxPooling1D()(c)
        convs.append(c)
    x1 = Concatenate()(convs)
    # title u
    inp_tu = Input(shape=(max_len_tu,))
    x2 = Embedding(embed_weights.shape[0], embed_weights.shape[1], weights=[embed_weights], trainable=False)(inp_tu)
    convs = []
    for kernel_size in [3, 4, 5]:
        c = Conv1D(128, kernel_size, activation='relu')(x2)
        c = GlobalMaxPooling1D()(c)
        convs.append(c)
    x2 = Concatenate()(convs)
    # desc q
    inp_dq = Input(shape=(max_len_dq,))
    y1 = Embedding(embed_weights.shape[0], embed_weights.shape[1], weights=[embed_weights], trainable=False)(inp_dq)
    convs = []
    for kernel_size in [3, 4, 5]:
        c = Conv1D(128, kernel_size, activation='relu')(y1)
        c = GlobalMaxPooling1D()(c)
        convs.append(c)
    y1 = Concatenate()(convs)
    # desc u
    inp_du = Input(shape=(max_len_du,))
    y2 = Embedding(embed_weights.shape[0], embed_weights.shape[1], weights=[embed_weights], trainable=False)(inp_du)
    convs = []
    for kernel_size in [3, 4, 5]:
        c = Conv1D(128, kernel_size, activation='relu')(y2)
        c = GlobalMaxPooling1D()(c)
        convs.append(c)
    y2 = Concatenate()(convs)
    ## dense
    x = Concatenate()([x1, x2, y1, y2])
    x = Dense(256)(x)
    x = BatchNormalization()(x)
    x = PReLU()(x)
    x = Dropout(0.2)(x)
    x = Dense(256)(x)
    x = BatchNormalization()(x)
    x = PReLU()(x)
    x = Dropout(0.2)(x)
    out = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=[inp_tq, inp_tu, inp_dq, inp_du], outputs=out)
    return model

In [21]:
train_tq, train_tu, train_dq, train_du = title_q[:len_train], title_u[:len_train], desc_q[:len_train], desc_u[:len_train]
test_tq, test_tu, test_dq, test_du = title_q[len_train:], title_u[len_train:], desc_q[len_train:], desc_u[len_train:]
train_y = data['label'].values[:len_train]

In [22]:
print(train_tq.shape)
print(test_tq.shape)
print(train_y.sum() / len(train_y))

(9489162, 38)
(1141683, 38)
0.1773515933229931


In [23]:
BATCH_SIZE = 1024

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for i, (tr_idx, va_idx) in enumerate(kfold.split(train_tq, train_y)):
    
    if i == 0:
        continue
    
    print('-'*100)
    print('Fold %d' % i)
    trn_tq, trn_tu, trn_dq, trn_du, trn_y = train_tq[tr_idx], train_tu[tr_idx], train_dq[tr_idx], train_du[tr_idx], train_y[tr_idx]
    val_tq, val_tu, val_dq, val_du, val_y = train_tq[va_idx], train_tu[va_idx], train_dq[va_idx], train_du[va_idx], train_y[va_idx]
    
    K.clear_session()
    model = Net()
    model.compile(loss='binary_crossentropy', optimizer='adam',)
    
    trn_generator = DataSequence(trn_tq, trn_tu, trn_dq, trn_du, trn_y, batch_size=BATCH_SIZE)
    val_generator = DataSequence(val_tq, val_tu, val_dq, val_du, val_y, batch_size=BATCH_SIZE)
    
    history = model.fit_generator(generator=trn_generator, 
                        epochs=5, 
                        verbose=1, 
                        callbacks=[MetricsCallback(trn_tq, trn_tu, trn_dq, trn_du, trn_y,
                                                   val_tq, val_tu, val_dq, val_du, val_y, 
                                                   batch_size=BATCH_SIZE*4, 
                                                   save_name='ques_user_title_desc_textcnn_enc_weight_fold%d.h5' % i)], 
                        max_queue_size=100, 
                        workers=1, 
                        use_multiprocessing=False,)
    
    del trn_tq, trn_tu, trn_dq, trn_du, trn_y
    del val_tq, val_tu, val_dq, val_du, val_y
    gc.collect()

----------------------------------------------------------------------------------------------------
Fold 1
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/5
7414/7414 [==============================] - 792s 107ms/step - loss: 0.4506
roc-auc: 0.6622 - roc-auc_val: 0.6536                                                                                                    
Epoch 2/5
7414/7414 [==============================] - 797s 108ms/step - loss: 0.4429
roc-auc: 0.6787 - roc-auc_val: 0.6604                                                                                                    
Epoch 3/5
7414/7414 [==============================] - 794s 107ms/step - loss: 0.4384
roc-auc: 0.6949 - roc-auc_val: 0.6606                                                                                                    
Epoch 4/5
7414/7414 [=

In [24]:
# del trn_tq, trn_tu, trn_dq, trn_du, trn_y
# del val_tq, val_tu, val_dq, val_du, val_y
# gc.collect()

In [25]:
# 0.4608

In [26]:
# BATCH_SIZE = 256

# kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [27]:
test_generator = DataSequence(test_tq, test_tu, test_dq, test_du, np.zeros(test_tq.shape[0]), batch_size=BATCH_SIZE)

test_pred = np.zeros((test_tq.shape[0], 1))
train_pred = np.zeros((train_tq.shape[0], 1))

for i, (tr_idx, va_idx) in enumerate(kfold.split(train_tq, train_y)):

    print('-'*100)
    print('Fold %d' % i)
    val_tq, val_tu, val_dq, val_du, val_y = train_tq[va_idx], train_tu[va_idx], train_dq[va_idx], train_du[va_idx], train_y[va_idx]

    K.clear_session()
    model = Net()
    model.compile(loss='binary_crossentropy', optimizer='adam',)

    val_generator = DataSequence(val_tq, val_tu, val_dq, val_du, val_y, batch_size=BATCH_SIZE)

    model.load_weights(os.path.join(WEIGHT_PATH, 'ques_user_title_desc_textcnn_enc_weight_fold%d.h5' % i))

    train_pred[va_idx] = model.predict_generator(val_generator, verbose=1)
    test_pred += model.predict_generator(test_generator, verbose=1) / 5
    
    print(roc_auc_score(val_y, train_pred[va_idx]))
    
    del val_tq, val_tu, val_dq, val_du, val_y
    gc.collect()
    print('Predict Done.')

----------------------------------------------------------------------------------------------------
Fold 0
1115/1115 [==============================] - 34s 30ms/step
0.6629150575338254
Predict Done.
----------------------------------------------------------------------------------------------------
Fold 1
1115/1115 [==============================] - 35s 32ms/step
0.6607267577740655
Predict Done.
----------------------------------------------------------------------------------------------------
Fold 2
1115/1115 [==============================] - 34s 31ms/step
0.6620717929307861
Predict Done.
----------------------------------------------------------------------------------------------------
Fold 3
1115/1115 [==============================] - 36s 32ms/step
0.6630800574045425
Predict Done.
----------------------------------------------------------------------------------------------------
Fold 4
1115/1115 [==============================] - 36s 32ms/step
0.6631191930953265
Predict Done.


In [28]:
arr = np.vstack([train_pred, test_pred])
feat = pd.DataFrame(arr, columns=['ques_user_title_desc_textcnn_enc'])
feat.head()

,ques_user_title_desc_textcnn_enc
0,0.113665
1,0.189822
2,0.061003
3,0.096835
4,0.296237


In [29]:
feat.to_pickle('./feats/ques_user_title_desc_textcnn_enc.pkl')

In [30]:
feat.shape

(10630845, 1)